In [1]:
# This script is used for interacting with OpenLCA.
# Codes are adapted from Julian Rickert @GreenDelta by Ao Chen @ETH.
# Connect to IPC server in OpenLCA software before running. (Port: 8080)

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('notebook', {'highlight_selected_word': {
    'highlight_across_all_cells': True,
    'only_cells_in_scroll': False,
    'delay': 500,
    'code_cells_only': True,
}})


import pandas as pd
import olca
import uuid
import math
from datetime import datetime

from matplotlib import pyplot as plt
import matplotlib.mlab as mlab
from matplotlib import rcParams
import matplotlib.patches as mpatches
import seaborn as sns

params = {'mathtext.default': 'regular' }

client = olca.Client(8080)
client

In [2]:
# Get all process dataset in database
process_descriptor = client.get_descriptors(olca.Process)
process_list = []
id_list = []

for process in process_descriptor:
    process_list.append(process.name)
    id_list.append(process.id)
processes_df = pd.DataFrame(list(zip(process_list,
                                   id_list)),
                               columns=['name', 'id'])
processes_df

,name,id
0,"heat production, heavy fuel oil, at industrial...",4f582f90-d0ae-3552-be43-70c730e82af0
1,"containerboard production, fluting medium, rec...",fdffa34a-17fa-3676-9941-0a8a2bc5f5ac
2,"market for wood chips, dry, measured as dry ma...",35ca5293-f067-326c-ae45-920013c734c7
3,"treatment of waste glass, unsanitary landfill,...",ab082ee7-7469-3a1f-8e8f-f71328b9e842
4,"heat and power co-generation, natural gas, con...",3c7c0ba3-acd4-3e19-be88-8fff9dc87788
...,...,...
19123,"steel production, electric, chromium steel 18/...",39b4f8c5-a396-3e00-b8c1-db951800e388
19124,"market for glued laminated timber, PUR-glue | ...",c8a2e84f-6e92-3caa-ab1f-388427902eec
19125,"electricity production, hard coal | electricit...",ea622ed5-24a9-3c50-adbb-6f8eb7465bff
19126,"land already in use, annual cropland to forest...",8a8f8368-b831-3525-a257-065cacf08853


In [4]:
# Search specific process databases with key word(s)
# Note that the search is blankspace-sensitive
search_process_df = processes_df[processes_df['name'].str.contains(r'heat production(?!$)')]
search_process_df.reset_index(drop=True, inplace=True)

search_process_df

,name,id
0,"heat production, heavy fuel oil, at industrial...",4f582f90-d0ae-3552-be43-70c730e82af0
1,"heat production, softwood chips from forest, a...",8a563013-8c78-387a-b578-6cfc5b903224
2,"heat production, untreated waste wood, at furn...",7bf9a3bf-0843-37d4-ad30-c3763c16c3b3
3,"heat production, untreated waste wood, at furn...",e94ac306-7d98-33f6-8b62-e91ac40977ec
4,"wheat production, Swiss integrated production,...",faa4e42c-ca9e-3fba-8e8d-8803e614d321
...,...,...
187,"heat production, softwood chips from forest, a...",6beac7b6-6f68-32aa-968f-3307b68c1496
188,"heat production, untreated waste wood, at furn...",fbe5bff5-a475-3f3e-8257-c266effb0e06
189,"heat production, untreated waste wood, at furn...",38dfe932-b941-3f9b-aad3-7f9928af97bf
190,"heat production, wood pellet, at furnace 9kW, ...",f335c734-e215-31e5-b8ea-c5eb406f2c0d


In [6]:
# Get all flow dataset in database
flow_descriptor = client.get_descriptors(olca.Flow)
flow_list = []
id_list = []

for flow in flow_descriptor:
    flow_list.append(flow.name)
    id_list.append(flow.id)
flows_df = pd.DataFrame(list(zip(flow_list,
                                   id_list)),
                               columns=['name', 'id'])
flows_df

,name,id
0,"industrial furnace, 1MW, oil",971337d4-7826-4ec6-bfef-5e5c6dd517be
1,"heat, district or industrial, other than natur...",71e2f1db-a2c5-44d0-8337-dfff15be974d
2,chimney,21663725-f3b4-4d6a-a45a-baf3cb343a6a
3,"oil storage, 3000l",9eb12779-1ac9-4b9a-8545-3dc3b14761ed
4,"electricity, low voltage",d69294d7-8d64-4915-a896-9996a014c410
...,...,...
5328,Silicon dioxide,46190ab6-8a25-40e2-8be1-b5145fed2f25
5329,Silicon tetrachloride,55f0f584-45ca-4bc3-af11-098164f471e6
5330,Ziram,fb4f9b12-7ec0-59aa-8087-c6115d1fcd31
5331,Propineb,8704d23b-dc6c-5fd8-94be-60e6fabb02f5


In [7]:
# Search specific flow databases with key word(s)
# Note that the search is blankspace-sensitive
search_flow_df = flows_df[flows_df['name'].str.contains(r'oil(?!$)')]
search_flow_df.reset_index(drop=True, inplace=True)

search_flow_df

,name,id
0,"oil storage, 3000l",9eb12779-1ac9-4b9a-8545-3dc3b14761ed
1,spoil from hard coal mining,5c772de8-0e4b-4150-a43d-d4a7668434dc
2,"waste textile, soiled",11b71553-cc51-438e-8cb5-0337836737ae
3,"nuclear power plant, boiling water reactor 1000MW",ae530eaa-1edd-49a7-915c-65d74a4554d7
4,"onshore well, oil/gas",02dfba07-7533-4d54-a1f3-478ae9314a09
5,"laminating service, foil, with acrylic binder",45d92845-1234-4ab6-8401-2e2c0018d13a
6,"Carbon dioxide, from soil or biomass stock",78eb1859-abd9-44c6-9ce3-f3b5b33d619c
7,"oil power plant, 500MW",9aecb33f-2000-44eb-8d87-724bd1b8f1c6
8,"zinc coat, coils",ec750cf4-a48f-4f2d-b40f-0c2f2f4cb6a0
9,"Carbon, organic, in soil or biomass stock",8c2fe757-6866-4ed2-9f89-81012ad774a0
